### Install lib 

In [3]:
%pip install meteostat pandas xlrd datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [meteostat]face]
Note: you may need to restart the kernel to use updated packages.


#### Data meteostat: Historical weather data of 8 departments in ile de france 


In [4]:
from meteostat import Point, Daily,Hourly
from datetime import datetime
import pandas as pd
import numpy as np

## Meteostat Data structure <br>
#### time : The datetime of the observation <br>
 temp : The air temperature in celsius <br>
 dwpt : the dew point in Celsius <br>
 rhum : The relative humidity in percent (%) <br>
 prcp : The one hour precipitation total in mm <br>
 snow : The snow depth in mm <br>
 wdir : The average wind direction in degrees (°) <br>
 wspd : The average wind speed in km/h <br>
 wpgt : The peak wind gust in km/h <br>
 pres : The average sea-level air pressure in hPa <br>
 tsun : The one hour sunshine total in minutes (m) <br>
 coco : The [weather condition code ](https://dev.meteostat.net/formats.html#weather-condition-codes)

In [33]:
# Define 8 coordinates
idf_points = {
    "Paris": (48.8566, 2.3522),
    "Hauts-de-Seine": (48.8924, 2.2060), # Nanterre
    "Seine-Saint-Denis": (48.9109, 2.4395), # Bobigny
    "Val-de-Marne": (48.7904, 2.4556), #Créteil
    "Yvelines": (48.8049, 2.1204), # Versailles
    "Essonne": (48.6290, 2.4410), # Evry
    "Val-d'Oise ": (49.0360, 2.0630), # Cergy
    "Seine-et-Marne": (48.5390, 2.6550) #Melun
}

# Define time range
start = datetime(2024, 1, 1)
end = datetime(2024, 12, 31)

# Retrieve daily data for each city
dfs = []
for city, (lat, lon) in idf_points.items():
    p = Point(lat, lon)
    data = Daily(p, start, end).fetch().reset_index()
    data['city'] = city
    dfs.append(data)

weather_all = pd.concat(dfs, ignore_index=True)
weather_all



,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,city
0,2024-01-01,8.2,6.2,10.9,4.1,<NA>,<NA>,16.4,46.0,1009.3,93.0,Paris
1,2024-01-02,10.9,9.9,11.8,12.2,<NA>,<NA>,28.0,83.0,998.6,0.0,Paris
2,2024-01-03,10.8,9.1,12.7,6.2,<NA>,<NA>,20.2,65.0,999.8,108.0,Paris
3,2024-01-04,9.6,7.6,11.1,0.0,<NA>,<NA>,16.4,46.0,1006.2,137.0,Paris
4,2024-01-05,7.9,6.1,9.9,0.2,<NA>,<NA>,12.2,48.0,1004.3,253.0,Paris
...,...,...,...,...,...,...,...,...,...,...,...,...
2923,2024-12-27,1.3,-1.3,4.9,0.2,<NA>,<NA>,5.0,18.0,1032.9,319.0,Seine-et-Marne
2924,2024-12-28,-0.7,-1.8,0.8,0.2,<NA>,<NA>,8.2,35.0,1031.4,40.0,Seine-et-Marne
2925,2024-12-29,1.9,0.4,3.3,0.0,<NA>,<NA>,7.4,21.0,1032.3,0.0,Seine-et-Marne
2926,2024-12-30,2.2,0.8,3.3,0.0,<NA>,<NA>,8.1,22.0,1032.6,22.0,Seine-et-Marne


In [34]:
weather_all.isnull().sum()

time       0
tavg       0
tmin       0
tmax       0
prcp      12
snow    2893
wdir    2928
wspd       0
wpgt     366
pres       0
tsun     366
city       0
dtype: int64

In [35]:
weather_all.describe()

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
count,2928,2928.0,2928.0,2928.0,2916.0,35.0,0.0,2928.0,2562.0,2928.0,2562.0
mean,2024-07-01 12:00:00.000000256,12.602459,9.084665,16.380089,2.305178,1.0,<NA>,12.472643,38.385051,1016.244433,190.552693
min,2024-01-01 00:00:00,-3.1,-6.9,-2.0,0.0,1.0,<NA>,3.9,13.0,985.0,0.0
25%,2024-04-01 00:00:00,8.5,5.6,11.2,0.0,1.0,<NA>,8.5,28.0,1010.7,36.0
50%,2024-07-01 12:00:00,12.5,9.2,16.3,0.2,1.0,<NA>,11.4,35.0,1016.4,150.0
75%,2024-10-01 00:00:00,16.9,13.0,21.6,2.4,1.0,<NA>,15.2,46.0,1022.6,299.0
max,2024-12-31 00:00:00,28.8,22.0,37.0,73.6,1.0,<NA>,39.7,91.0,1036.6,735.0
std,NaN,6.017009,5.384032,7.044272,5.474174,0.0,<NA>,5.232901,14.012413,9.844798,172.85244


#### Drop snow, tsun


In [36]:
feature_meteo = ["time","temp","dwpt","rhum","prcp","wdir","wspd","pres","coco","city"]


In [37]:
# df_weather = weather_all[feature_meteo]

#### Data electricity ile de france

In [10]:
df_ele = pd.read_csv( "/home/onyxia/work/Python_Project_2A/data/RTE/data_ele-2020-2024.csv", low_memory=False)  
df_ele.head()

,Périmètre,Nature,Date,Heures,Consommation
0,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,00:00,8911.0
1,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,00:30,8814.0
2,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,01:00,8589.0
3,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,01:30,8575.0
4,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,02:00,8560.0


In [11]:
df_ele['Datetime'] = pd.to_datetime(
    df_ele['Date'].astype(str) + ' ' + df_ele['Heures'].astype(str),
    errors='coerce'
)
df_ele.head()

,Périmètre,Nature,Date,Heures,Consommation,Datetime
0,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,00:00,8911.0,2020-01-01 00:00:00
1,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,00:30,8814.0,2020-01-01 00:30:00
2,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,01:00,8589.0,2020-01-01 01:00:00
3,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,01:30,8575.0,2020-01-01 01:30:00
4,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,02:00,8560.0,2020-01-01 02:00:00


In [7]:
df_ele.shape

(1052352, 6)

In [41]:
# # Rename colunms ele
# ele = df.rename(columns = {"Périmètre":"Perimeter",
#                             "Nucléaire":"Nuclear",
#                             "Bioénergies":"Bioenergies",
#                             "Déstockage batterie ":"Battery clearance",
#                             "TCO Nucléaire (%)":"TCO Nuclear (%)",
#                             "TCH Nucléaire (%)":"TCH Nuclear",
#                             "TCO Bioénergies (%)":"TCO Bioenergies (%)",
#                             "TCH Bioénergies(%)":"TCh Bioenergies (%)"})

In [12]:
df_ele = (df_ele
          .sort_values('Datetime')
          .set_index('Datetime'))
# df_ele.index = pd.to_datetime(df_ele.index, errors='coerce')
df_ele_daily = (df_ele.resample('24h').agg({'Consommation': 'sum'}).reset_index())

In [14]:
df_ele_daily.shape

(1827, 2)

#### source population : https://idf.handidonnees.fr/page/population

In [43]:
population_idf = {
    "Paris": 2048472,
    "Hauts-de-Seine": 1663974,
    "Seine-Saint-Denis": 1711876,
    "Val-de-Marne": 1426433,
    "Yvelines": 1489363,
    "Essonne": 1343527,
    "Val-d'Oise": 1287589,
    "Seine-et-Marne": 1479615
}
for ele,items in population_idf.items():
    print(ele,items)

Paris 2048472
Hauts-de-Seine 1663974
Seine-Saint-Denis 1711876
Val-de-Marne 1426433
Yvelines 1489363
Essonne 1343527
Val-d'Oise 1287589
Seine-et-Marne 1479615


#### In the reason of disable of data electricity in each department of ile de france, so we need to apply a method that we can obtain from an approximate of utilization of each department base on the each number population of the level aggregation in the IDF. Hence we have the method as follow: 
#### Let: <br>
&nbsp; $E_{d,t}$ &nbsp; is an utilization of the electricity in department d at the ti me t. <br>
&nbsp;   $E_{IDF,t}$ &nbsp; is an utilization of the electricity in IDF at the time t.  <br>
&nbsp;  $Pop_d$ &nbsp; is number of population in departement d. <br>

$$E_{d,t} = E_{IDF,t} \times \frac{Pop_d}{\sum_d{Pop_d}}$$

In [ ]:
# total_pop = sum(population_idf.values())

In [45]:
# create new columns cosommation electricity base on population in each department 
# ele_dept = (
#     pd.DataFrame({
#         'time': pd.Series(ele_hourly['time'].values.repeat(len(population_idf))),
#         'Department': list(population_idf.keys()) * len(ele_hourly),
#     })
#     .merge(ele_hourly[['time','Consommation']], on='time', how='left')
# )

# ele_dept['pop_weight'] = ele_dept['Department'].map(population_idf) / total_pop
# ele_dept['Electricity_MW'] = ele_dept['Consommation'] * ele_dept['pop_weight']
# ele_dept = ele_dept.drop(columns=['pop_weight'])
# ele_dept


In [46]:
# weather_name_map = {
#     'Hauts-de-Seine':'Hauts-de-Seine',
#     'Seine-Saint-Denis':'Seine-Saint-Denis',
#     'Val-de-Marne':'Val-de-Marne',
#     'Yvelines':'Yvelines',
#     'Essonne':'Essonne',
#     "Val-d'Oise":"Val-d'Oise",
#     'Seine-et-Marne':'Seine-et-Marne'
# }
# weather_long = weather_all.copy()
# weather_long['Department'] = weather_long['city'].map(weather_name_map)
# weather_long = weather_long.dropna(subset=['Department'])

# df_dep = (
#     weather_long
#     .merge(ele_dept[['time','Department','Electricity_MW']], on=['time','Department'], how='inner')
#     .sort_values(['Department','time']).reset_index()
# )
# df_dep = df_dep.drop(columns=['index','snow','city'])
# df_dep


In [47]:
# df_dep.isnull().sum()

In [48]:
# df_dep.describe()

In [49]:
df_weather = pd.read_csv("/home/onyxia/work/Python_Project_2A/data/weather/idf_weather_daily_power_2020_2024.csv")
df_weather.head(2)

,date,year,month,day,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,GWETROOT,GWETTOP,PRECTOTCORR,PS,...,RH2M,T2M,T2M_MAX,T2M_MIN,T2M_RANGE,WD10M,WS10M,WS2M,insee_dep,department
0,2020-01-01,2020,1,1,1.79,5.19,0.58,0.6,0.68,101.91,...,98.50,1.95,3.86,0.25,3.61,160.8,3.07,2.06,75,Paris
1,2020-01-02,2020,1,2,2.09,4.76,0.58,0.6,0.68,101.39,...,98.45,4.84,6.70,2.63,4.07,199.1,4.20,2.84,75,Paris


In [50]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14616 entries, 0 to 14615
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               14616 non-null  object 
 1   year               14616 non-null  int64  
 2   month              14616 non-null  int64  
 3   day                14616 non-null  int64  
 4   ALLSKY_SFC_SW_DWN  14616 non-null  float64
 5   CLRSKY_SFC_SW_DWN  14616 non-null  float64
 6   GWETROOT           14616 non-null  float64
 7   GWETTOP            14616 non-null  float64
 8   PRECTOTCORR        14616 non-null  float64
 9   PS                 14616 non-null  float64
 10  QV2M               14616 non-null  float64
 11  RH2M               14616 non-null  float64
 12  T2M                14616 non-null  float64
 13  T2M_MAX            14616 non-null  float64
 14  T2M_MIN            14616 non-null  float64
 15  T2M_RANGE          14616 non-null  float64
 16  WD10M              146

In [51]:
df_weather[['date']].values

array([['2020-01-01'],
       ['2020-01-02'],
       ['2020-01-03'],
       ...,
       ['2024-12-29'],
       ['2024-12-30'],
       ['2024-12-31']], shape=(14616, 1), dtype=object)

In [52]:
pop_idf = pd.read_csv("/home/onyxia/work/Python_Project_2A/data/population/population_idf-1968-2024.csv")
pop_idf.tail()

,Annee,Paris,Seine-et-Marne,Yvelines,Essonne,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne,Val-d’Oise,Île-de-France
33,2020,2 145 906,1 428 636,1 449 723,1 306 118,1 626 213,1 655 422,1 407 972,1 251 804,12 271 794
34,2021,2 133 111,1 438 100,1 456 365,1 313 768,1 635 291,1 668 670,1 415 367,1 256 607,12 317 279
35,2022,2 113 705,1 452 399,1 470 778,1 324 546,1 647 435,1 681 725,1 419 531,1 270 845,12 380 964
36,2023,2 087 600,1 464 800,1 473 700,1 331 800,1 651 400,1 701 000,1 433 900,1 275 700,12 419 900
37,2024,2 104 300,1 455 800,1 467 800,1 325 900,1 646 400,1 690 400,1 428 100,1 269 700,12 388 400


In [53]:
df_ele_daily.Datetime

0      2020-01-01
1      2020-01-02
2      2020-01-03
3      2020-01-04
4      2020-01-05
          ...    
1822   2024-12-27
1823   2024-12-28
1824   2024-12-29
1825   2024-12-30
1826   2024-12-31
Name: Datetime, Length: 1827, dtype: datetime64[ns]

In [54]:
#select only last 5 years 2020-2025 from population 
dept_cols = ['Paris','Hauts-de-Seine','Seine-Saint-Denis','Val-de-Marne',
             'Yvelines','Essonne',"Val-d’Oise",'Seine-et-Marne']
pop = pop_idf.query('Annee >= 2020 and Annee <= 2024').copy()
for c in dept_cols:
    pop[c] = (pop[c].astype(str)
                    .str.replace(r'[^\d]', '', regex=True)
                    .replace('', np.nan).astype('float').astype('Int64'))

pop_long = (pop.melt(id_vars='Annee', value_vars=dept_cols,
                     var_name='Department', value_name='Population')
               .dropna())
pop_long['year'] = pop_long['Annee'].astype(int)
pop_long['Population'] = pop_long['Population'].astype(int)
pop_long['year_pop_total'] = pop_long.groupby('year')['Population'].transform('sum')
pop_long['Weight'] = pop_long['Population'] / pop_long['year_pop_total']
weights = pop_long[['year','Department','Population','Weight']]
pop_long.head()

,Annee,Department,Population,year,year_pop_total,Weight
0,2020,Paris,2145906,2020,12271794,0.174865
1,2021,Paris,2133111,2021,12317279,0.173180
2,2022,Paris,2113705,2022,12380964,0.170722
3,2023,Paris,2087600,2023,12419900,0.168085
4,2024,Paris,2104300,2024,12388400,0.169861


In [ ]:
#Prep daily regional electricity
ele_daily = df_ele_daily.copy()
ele_daily['time'] = pd.to_datetime(ele_daily['Datetime'])
ele_daily['year'] = ele_daily['time'].dt.year
ele_daily = ele_daily[['time','year','Consommation']]

#Split by population weights 
ele_dept_daily = (ele_daily.merge(weights, on='year', how='inner')
                            .assign(Electricity_MW=lambda d: d['Consommation'] * d['Weight'])
                            [['time','Department','Consommation','Electricity_MW']]
                            .sort_values(['time','Department'])
                            .reset_index(drop=True))

ele_dept_daily.head()


,time,Department,Consommation,Electricity_MW
0,2020-01-01,Essonne,472417.0,50280.533328
1,2020-01-01,Hauts-de-Seine,472417.0,62602.963089
2,2020-01-01,Paris,472417.0,82609.150284
3,2020-01-01,Seine-Saint-Denis,472417.0,63727.397557
4,2020-01-01,Seine-et-Marne,472417.0,54997.006404


In [56]:
# Prepare weather DataFrame
df_weather['date'] = pd.to_datetime(df_weather['date'])   
df_weather.rename(columns={'date': 'time', 'department': 'Department'}, inplace=True)
df_weather.head()

,time,year,month,day,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,GWETROOT,GWETTOP,PRECTOTCORR,PS,...,RH2M,T2M,T2M_MAX,T2M_MIN,T2M_RANGE,WD10M,WS10M,WS2M,insee_dep,Department
0,2020-01-01,2020,1,1,1.79,5.19,0.58,0.6,0.68,101.91,...,98.50,1.95,3.86,0.25,3.61,160.8,3.07,2.06,75,Paris
1,2020-01-02,2020,1,2,2.09,4.76,0.58,0.6,0.68,101.39,...,98.45,4.84,6.70,2.63,4.07,199.1,4.20,2.84,75,Paris
2,2020-01-03,2020,1,3,2.69,5.16,0.58,0.6,1.01,101.30,...,94.04,6.97,11.43,1.89,9.54,245.7,6.39,4.41,75,Paris
3,2020-01-04,2020,1,4,4.69,5.51,0.58,0.6,0.18,102.34,...,94.54,3.88,8.65,0.15,8.50,297.1,3.43,2.08,75,Paris
4,2020-01-05,2020,1,5,3.75,5.69,0.58,0.6,0.11,102.39,...,92.34,4.10,8.27,-0.92,9.19,207.4,1.75,0.99,75,Paris


In [57]:
df_final = pd.merge(ele_dept_daily,df_weather, on=['time','Department'],how='inner')
df_final

,time,Department,Consommation,Electricity_MW,year,month,day,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,GWETROOT,...,QV2M,RH2M,T2M,T2M_MAX,T2M_MIN,T2M_RANGE,WD10M,WS10M,WS2M,insee_dep
0,2020-01-01,Essonne,472417.0,50280.533328,2020,1,1,1.79,5.19,0.65,...,4.59,98.63,2.83,5.05,0.91,4.14,166.1,2.83,1.84,91
1,2020-01-01,Hauts-de-Seine,472417.0,62602.963089,2020,1,1,1.79,5.19,0.58,...,4.25,98.50,1.95,3.86,0.25,3.61,160.8,3.07,2.06,92
2,2020-01-01,Paris,472417.0,82609.150284,2020,1,1,1.79,5.19,0.58,...,4.25,98.50,1.95,3.86,0.25,3.61,160.8,3.07,2.06,75
3,2020-01-01,Seine-Saint-Denis,472417.0,63727.397557,2020,1,1,1.79,5.19,0.58,...,4.25,98.50,1.95,3.86,0.25,3.61,160.8,3.07,2.06,93
4,2020-01-01,Seine-et-Marne,472417.0,54997.006404,2020,1,1,1.79,5.19,0.73,...,4.14,98.36,1.50,3.68,-0.51,4.19,162.0,3.03,2.03,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12784,2024-12-31,Paris,482912.0,82027.680863,2024,12,31,1.59,5.52,0.66,...,3.76,95.07,0.76,2.94,-1.07,4.01,205.7,5.42,3.77,75
12785,2024-12-31,Seine-Saint-Denis,482912.0,65893.452326,2024,12,31,1.59,5.52,0.66,...,3.76,95.07,0.76,2.94,-1.07,4.01,205.7,5.42,3.77,93
12786,2024-12-31,Seine-et-Marne,482912.0,56748.513900,2024,12,31,1.59,5.52,0.79,...,3.56,95.40,-0.04,2.44,-1.99,4.43,204.8,4.45,3.09,77
12787,2024-12-31,Val-de-Marne,482912.0,55668.740693,2024,12,31,1.59,5.52,0.66,...,3.76,95.07,0.76,2.94,-1.07,4.01,205.7,5.42,3.77,94


In [58]:
df_final.columns 

Index(['time', 'Department', 'Consommation', 'Electricity_MW', 'year', 'month',
       'day', 'ALLSKY_SFC_SW_DWN', 'CLRSKY_SFC_SW_DWN', 'GWETROOT', 'GWETTOP',
       'PRECTOTCORR', 'PS', 'QV2M', 'RH2M', 'T2M', 'T2M_MAX', 'T2M_MIN',
       'T2M_RANGE', 'WD10M', 'WS10M', 'WS2M', 'insee_dep'],
      dtype='object')